Run cells bellow to download and preprocess the **MovieLens20M** dataset.

Estimated time is around 45 minutes.

In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip

--2023-05-04 09:18:08--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  1.62MB/s    in 1m 40s  

2023-05-04 09:19:48 (1.90 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [3]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [4]:
import pandas as pd

All Items

In [5]:
m = pd.read_csv('ml-20m/movies.csv')
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.apply(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
items.to_json('items.json')

In [6]:
items

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


Purchases and grouped purchases.

In [7]:
interactions = pd.read_csv('ml-20m/ratings.csv')
interactions = interactions[interactions.rating>=4.]
interactions = interactions.sort_values(['userId','timestamp'])
interactions['itemid'] = interactions['movieId'].apply(str)
interactions['userid'] = interactions['userId'].apply(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid','userid','amount','date']]
interactions.to_json("purchases.json")
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
iii.to_json('purchases_txt.json')

In [8]:
interactions

,userId,movieId,rating,timestamp,itemid,userid,amount,date,itemids
23,1,1079,4.0,1094785665,1079,1,1,1094785665,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
96,1,2959,4.0,1094785698,2959,1,1,1094785698,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
111,1,3996,4.0,1094785727,3996,1,1,1094785727,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
6,1,151,4.0,1094785734,151,1,1,1094785734,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
58,1,1374,4.0,1094785746,1374,1,1,1094785746,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
...,...,...,...,...,...,...,...,...,...
19999942,138493,823,4.5,1260209589,823,138493,1,1260209589,"3174,2872,48780,2662,1566,2857,2367,3996,7090,..."
20000257,138493,68319,4.5,1260209720,68319,138493,1,1260209720,"3174,2872,48780,2662,1566,2857,2367,3996,7090,..."
20000213,138493,40819,4.5,1260209726,40819,138493,1,1260209726,"3174,2872,48780,2662,1566,2857,2367,3996,7090,..."
20000200,138493,33794,4.0,1260209811,33794,138493,1,1260209811,"3174,2872,48780,2662,1566,2857,2367,3996,7090,..."


Keep only users with 5 or more interactions.

Purchases.

In [9]:
purchases=pd.read_json('purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
purchases_pu5.to_json('purchases_pu5.json')

In [10]:
purchases_item_count_pu5

,itemid,nr_of_purchases
6620,318,55705
6186,296,52265
12160,593,50061
7545,356,47302
4894,260,42578
...,...,...
3318,131050,1
3317,131027,1
3316,131023,1
3313,130996,1


Grouped purchases.

In [11]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json('purchases_txt_pu5.json')

/tmp/ipykernel_1026/126306790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))


In [12]:
iii

,userid,itemids
23,1,"1079,2959,3996,151,1374,2291,4306,1214,1278,12..."
176,2,"62,1210,1356,1974,2951,110,589,1259,2948,3513,..."
257,3,"589,1721,2118,2710,2722,2448,2505,2694,858,122..."
424,4,"10,356,454,480,589,377,586,350,368,370,594,520..."
455,5,"62,141,736,780,671,832,1042,1393,150,590,380,4..."
...,...,...
19999614,138489,"1275,1747,3671,1061,2021,29,318,858,50,2019,91..."
19999644,138490,"111,3217,3654,3897,1535,2589,199,277,1735,1280..."
19999790,138491,"1093,4128,3186,2857,6874"
19999841,138492,"2115,908,58,2599,1500,1358,1288,1732,1252,1234..."


Users.

In [13]:
iii['userid'].to_frame().to_json('users_pu5.json')

In [14]:
iii['userid']

23               1
176              2
257              3
424              4
455              5
             ...  
19999614    138489
19999644    138490
19999790    138491
19999841    138492
20000071    138493
Name: userid, Length: 136677, dtype: object

Items

In [15]:
items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")

Items sorted by number of interactions

In [16]:
purchases_item_count_pu5.to_json("items_sorted_pu5.json")

In [17]:
purchases_item_count_pu5

,itemid,nr_of_purchases
6620,318,55705
6186,296,52265
12160,593,50061
7545,356,47302
4894,260,42578
...,...,...
3318,131050,1
3317,131027,1
3316,131023,1
3313,130996,1


Users sorted by number of interactions

In [18]:
pu5.to_json("users_sorted_pu5.json")

In [19]:
pu5

,userid,nr_of_purchases
28626,125794,3177
118782,82418,2655
120595,8405,2503
20206,118205,2377
35395,131894,2349
...,...,...
4572,104116,5
28224,125431,5
11086,109989,5
67106,3584,5


Create train, val and test split.

In [20]:
users = pd.read_json('users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

test_users.to_json("test_users.json")
val_users.to_json("val_users.json")
train_users.to_json("train_users.json") 

len(train_users),len(val_users),len(test_users)

(116677, 10000, 10000)

In [21]:
train_users

,userid
9141029,63177
17703308,122423
5668357,39001
1562218,10557
2448694,16563
...,...
6557517,45078
7804590,53850
56705,412
937183,6291


List of generated json files with preprocessed data.

In [22]:
!ls *.json

items.json	       purchases_pu5.json      train_users.json
items_pu5.json	       purchases_txt.json      users_pu5.json
items_sorted_pu5.json  purchases_txt_pu5.json  users_sorted_pu5.json
purchases.json	       test_users.json	       val_users.json
